In [6]:
%env CUDA_VISIBLE_DEVICES=2
%env TOKENIZERS_PARALLELISM=false

env: CUDA_VISIBLE_DEVICES=2
env: TOKENIZERS_PARALLELISM=false


In [1]:
from datasets import load_dataset
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import json

# HuggingFace IWSLT Data 

In [2]:
def build_dictionary_from_xml(xml_filepath_list, lang):
    """
    Build a dictionary for combined xml files of a language
    """
    
    files_list = []
    for fp in xml_filepath_list:
        with open(fp, 'r', encoding="utf-8") as f: # , 
            data = f.read()
        bs_data = BeautifulSoup(data, "xml")
        docs = bs_data.find_all('doc')
        files_list.append(docs)

    docs_dict = {}
    for docs in files_list:
        for doc in docs:
            #docid = int(doc['docid'])
            talkid = int(doc.find('talkid').get_text())
            seg_elements = doc.find_all('seg')
            segs = [seg.get_text(strip=True) for seg in seg_elements]
            langs_to_segs = {} 
            langs_to_segs[lang] = segs
            docs_dict[f"talk_id {talkid}"] = langs_to_segs
    
    return docs_dict



In [9]:
parallel_lang_list = [["en", "ja"],["en", "ar"], ["en",  "fr"], ["en", "de"], ["en", "ko"], ["en", "zh"]]
rel_path = "/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/"


for parallel in parallel_lang_list:
    parallel_lang_dicts = []
    tgt_lang = parallel[1]
    print (tgt_lang)
    for lang in parallel:
        xml_filepath_list = [
            rel_path+f"{tgt_lang}-en/IWSLT17.TED.dev2010.{tgt_lang}-en.{lang}.xml",
            rel_path+f"{tgt_lang}-en/IWSLT17.TED.tst2010.{tgt_lang}-en.{lang}.xml",
            rel_path+f"{tgt_lang}-en/IWSLT17.TED.tst2011.{tgt_lang}-en.{lang}.xml",
            rel_path+f"{tgt_lang}-en/IWSLT17.TED.tst2012.{tgt_lang}-en.{lang}.xml",
            rel_path+f"{tgt_lang}-en/IWSLT17.TED.tst2013.{tgt_lang}-en.{lang}.xml",
            rel_path+f"{tgt_lang}-en/IWSLT17.TED.tst2014.{tgt_lang}-en.{lang}.xml",
            rel_path+f"{tgt_lang}-en/IWSLT17.TED.tst2015.{tgt_lang}-en.{lang}.xml",
                           ]
        docs_dict = build_dictionary_from_xml(xml_filepath_list, lang)
        parallel_lang_dicts.append(docs_dict)
    shared_talk_ids = get_shared_talkids(lang_dicts=parallel_lang_dicts)
    parallel_talks_dict = combine_langs_into_one_dict(lang_dicts = parallel_lang_dicts, shared_talk_ids=shared_talk_ids)
    with open(f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/ted_{parallel[0]}-{parallel[1]}", "w", encoding="utf-8") as outfile:
        json.dump(parallel_talks_dict, outfile, indent=4, ensure_ascii=False)

ja
ar
fr
de
ko
zh


In [11]:
lang_list = ["ar", "fr", "zh", "ko", "de", "ja"]
lang_dicts = []
lang_dicts.append(docs_dict_en)
for lang in lang_list:
    xml_filepath_list = [f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/{lang}-en/IWSLT17.TED.dev2010.{lang}-en.{lang}.xml",
                        f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/{lang}-en/IWSLT17.TED.tst2010.{lang}-en.{lang}.xml",
                        f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/{lang}-en/IWSLT17.TED.tst2011.{lang}-en.{lang}.xml",
                        f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/{lang}-en/IWSLT17.TED.tst2012.{lang}-en.{lang}.xml",
                        f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/{lang}-en/IWSLT17.TED.tst2013.{lang}-en.{lang}.xml",
                        f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/{lang}-en/IWSLT17.TED.tst2014.{lang}-en.{lang}.xml",
                        f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/{lang}-en/IWSLT17.TED.tst2015.{lang}-en.{lang}.xml",
                       ]
    docs_dict = build_dictionary_from_xml(xml_filepath_list, lang)
    lang_dicts.append(docs_dict)

In [4]:
def get_shared_talkids(lang_dicts):
    
    shared_keys = set(lang_dicts[0].keys())

    for i in lang_dicts[1:]:
        shared_keys.intersection_update(i.keys())

    return shared_keys

In [13]:
shared_talk_ids = get_shared_talkids(lang_dicts=lang_dicts)
print (shared_talk_ids)

{'talk_id 1366', 'talk_id 227', 'talk_id 1375', 'talk_id 1939', 'talk_id 1353', 'talk_id 1114', 'talk_id 2045', 'talk_id 1144', 'talk_id 1160', 'talk_id 1115', 'talk_id 1733', 'talk_id 1864', 'talk_id 783', 'talk_id 1659', 'talk_id 837', 'talk_id 1378', 'talk_id 1166', 'talk_id 1617', 'talk_id 1130', 'talk_id 1346', 'talk_id 1741', 'talk_id 2024', 'talk_id 69', 'talk_id 790', 'talk_id 1548', 'talk_id 1534', 'talk_id 1328', 'talk_id 1647', 'talk_id 1961', 'talk_id 1404', 'talk_id 2102', 'talk_id 1443', 'talk_id 1694', 'talk_id 1371', 'talk_id 2183', 'talk_id 792', 'talk_id 1829', 'talk_id 1104', 'talk_id 1755', 'talk_id 1922', 'talk_id 1932', 'talk_id 1634', 'talk_id 1330', 'talk_id 1373', 'talk_id 1381', 'talk_id 1368', 'talk_id 1997', 'talk_id 1858', 'talk_id 1337', 'talk_id 93', 'talk_id 1518', 'talk_id 453', 'talk_id 1382', 'talk_id 1553', 'talk_id 129', 'talk_id 799', 'talk_id 1161', 'talk_id 1898', 'talk_id 1359', 'talk_id 1736', 'talk_id 767', 'talk_id 1685', 'talk_id 2007', 'tal

In [6]:
def combine_langs_into_one_dict(lang_dicts, shared_talk_ids):
    
    _multilingual_talks_dict = {}

    for talk_id in shared_talk_ids:
        talk_dict = {}
        talk_dict[talk_id] = {}
        
        for lang_dict in lang_dicts:
            talk_dict[talk_id].update(lang_dict[talk_id])
        
        _multilingual_talks_dict.update(talk_dict)
                
    return _multilingual_talks_dict
                

In [ ]:
# Check if each talk has same number of segments (sentences)

for talk_id, lang_dict in multilingual_talks_dict.items():
    seg_count_list = []
    for lang, seg_list in lang_dict.items():
        seg_count_list.append(len(seg_list))
    #print (seg_count_list)
    if len(set(seg_count_list)) > 1:
        print (seg_count_list)

In [20]:
with open("/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/ted_multilingual", "w", encoding="utf-8") as outfile:
    json.dump(multilingual_talks_dict, outfile, indent=4, ensure_ascii=False)

In [ ]:
"""
for talk_id, lang_dict in multilingual_talks_dict.items():
    seg_count_list = []
    for lang, seg_list in lang_dict.items():
        seg_count_list.append(len(seg_list))
    #print (seg_count_list)
    if len(set(seg_count_list)) > 1:
        print (seg_count_list)
"""

In [11]:
zh_df = pd.read_json(f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/ted_en-ko")
zh_df = zh_df.transpose()

for en_talk, zh_talk in zip (zh_df["en"], zh_df["ko"]):
    print (len(en_talk), len(zh_talk))


55 55
82 82
35 35
64 64
75 75
94 94
116 116
82 82
57 57
101 101
59 59
76 76
164 164
174 174
68 68
14 14
30 30
151 151
70 70
204 204
191 191
80 80
228 228
96 96
57 57
110 110
34 34
26 26
32 32
52 52
127 127
82 82
94 94
171 171
126 126
67 67
117 117
89 89
43 43
63 63
78 78
65 65
120 120
148 148
88 88
102 102
110 110
32 32
118 118
132 132
77 77
91 91
147 147
51 51
41 41
28 28
63 63
109 109
92 92
230 230
59 59
60 60
72 72
81 81
48 48
47 47
74 74
83 83
175 175
149 149
123 123
39 39
75 75
67 67
293 293
60 60
42 42
154 154
59 59
117 117
60 60
104 104
222 222
152 152
97 97
91 91
40 40
169 169
125 125
59 59
129 129
77 77
198 198
176 176
42 42
51 51
146 146


In [11]:
file_list = ["en-ja", "en-ar", "en-fr", "en-zh", "en-ko", "en-zh"]

for i in file_list:
    df = pd.read_json(f"/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/ted_{i}")
    df = df.transpose()

In [12]:
df

,en,zh
talk_id 1404,"[This may sound strange, but I'm a big fan of...","[听起来可能有点怪， 我是个水泥砖的狂热爱好者。, 第一块水泥砖制造于1868年 是源于一个..."
talk_id 1160,"[This is a representation of your brain, and ...","[这是大脑的描绘图。 你的大脑可以被分为两个部分。, 左脑，主要负责理性方面， 而右脑， 主..."
talk_id 1096,"[Back in New York, I am the head of developmen...","[在纽约，我是“罗宾汉”非营利机构 发展部门的主管, 在我消灭贫穷之余，我也灭火 我是志愿消..."
talk_id 1736,[Growing up in Taiwan as the daughter of a ca...,[我在台湾长大。 作为一位书法家的女儿， 我最珍贵的记忆之一 是妈妈为我展示了 汉字的形意之...
talk_id 837,"[Several years ago here at TED, Peter Skillman...",[几年前，在TED大会上， Peter Skillman 介绍了一个设计挑战 叫做“棉花糖挑...
...,...,...
talk_id 1337,[I think we have to do something about a piec...,"[我觉得在当今的医学文化中， 有一部份文化必须要开始改变。, 而这个改变可以从一个医生开始，..."
talk_id 93,[I'm going to talk to you about some stuff tha...,[我想在这里谈谈我这本书里的部分内容 希望它和你们已经听到过的一些事物 引起共鸣 如果有哪些...
talk_id 1477,"[When I was a child growing up in Maine, one ...",[当我在缅因州长大的时候， 一件我最喜欢做的事 是在海滩上找沙钱。 因为大人说沙钱会带来好运...
talk_id 1659,"[I have never, ever forgotten the words of my ...","[我永远也不会忘记我那在流亡中去世的祖母 对我说过的话： “孩子，反抗卡扎菲，与他抗争。, ..."


In [15]:
for lang in df.columns:
    sum_length = 0
    for talk in df[lang]:
        talk_length = len(talk)
        sum_length += talk_length
        #print ("sum", sum_length)
    print (lang, "talk length")
    print (sum_length/len(df))

en talk length
97.28260869565217
ar talk length
98.59782608695652
fr talk length
98.77173913043478
zh talk length
98.18478260869566
ko talk length
97.76086956521739
de talk length
97.27173913043478
ja talk length
97.28260869565217


In [36]:
for i, j in zip (df["ko"][0:1000], df["ja"][0:1000]):
    print (len(i), len(j))
    print (i, j)

70 68
['시작하기 전에 블록을 가지고 놀았던 어린 시절을 회상해 주시기 바랍니다.', '손을 뻗어 쥐고 집어들고 움직이는 법을 배우는 동안 여러분은 사실 사고하고 문제를 해결하는 방법을 배우게 되죠. 공간적 관계를 이해하고 조작하면서 말입니다.', '공간적 논리는 우리가 세계를 이해하는 방식과 그래서 저는 컴퓨터 과학자로서 물리적 대상과의 상호작용 방식에서 영감을 얻어서 조언자 패티와 공동 연구자 지반 칼라니시와 함께 이것에 대해 호기심을 갖기 시작했습니다. 우리가 컴퓨터를 사용할 때 디지털 손가락처럼 평면 데스크탑 위를 이동하는 하나의 마우스 커서 대신에 정보를 두 손으로, 물리적으로 쥐고', '원하는 방식으로 배열할 수 있으면 어떨까?', "너무 흥미로운 질문이라서 우리는 그 답을 찾기로 했고 '시프터블'을 개발하게 됐습니다.", "간단히 말해 '시프터블'은 인터액티브 컴퓨터입니다. 쿠키 정도 크기죠.", '손으로 움직일 수 있고 서로를 감지할 수 있고, 움직임을 감지할 수 있고, 모니터와 무선 라디오가 내장돼 있습니다.', '가장 중요한 것은 물리적이라는 것이죠. 블록처럼 잡아서 움직일 수 있습니다.', '시프터블은 디지털 정보를 조작하는 새로운 생태계의 한 예입니다.', '이런 도구들이 더 물리적으로 변하고, 움직임을 인식하고 서로를 인식하고 우리가 그것들을 어떻게 움직이는지 정교하게 인식하게 되면 새롭고 재미있는 상호작용 방식의 탐구가 시작됩니다.', '간단한 예를 가지고 시작하겠습니다.', "이 '시프터블'은 비디오를 출력하기로 설정돼 있습니다. 한쪽으로 기울이면 비디오가 이쪽으로 기울어지고 반대로 기울이면 비디오도 돌아옵니다.", '이 인터액티브 초상화들은 서로를 인식합니다.', '나란히 두면 서로에게 흥미가 생기죠.', '둘러싸이면 그것도 감지합니다. 조금 당황할 지도 모르죠.', '움직임과 기울기도 감지할 수 있습니다.', '상호작용이 갖는 흥미로운 의의 중 하나는 일상적인 동작을 데이터에 적용하는 것임을 깨달았습니다. 예를 들어 액체를 

# Original XML Data Preprocessing

### Preprocessing Rule
1. extract talk id and (content + file id) separately
2.  Check the length of both
    - If same length:
        - Concatenate both
    - If different length:
        - Delete duplicates in content df or add corresponding talkid refer to other languages.  
        - When same length, concatenate both. 
3. Sort the concatenated df by talkid
4. Check if the parallelism work in two languages 
5. Delete row where the other language file doesn't have talk id

In [4]:
from datasets import load_dataset
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [196]:
languages = ["ja", "en", "de", "ko", "zh", "zh-cn", "zh-tw", "ar", "fr"]

def create_talkid_df(languages):
    talkid_dfs = {}
    print ("The number of talk ids in each languages: ")
    for lang in languages:
        with open(f'/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_{lang}.xml', 'r', encoding="utf-8") as f: # , 
            data = f.read()
            bs_data = BeautifulSoup(data, "xml")
            talkid = bs_data.find_all("talkid")
            print (f"{lang}:", len(talkid))
            talkid_df = pd.DataFrame({"talk_id":talkid})
            talkid_df["talk_id"]=talkid_df["talk_id"].map(lambda x:str(x))
            talkid_df["talk_id"]=talkid_df["talk_id"].map(lambda x:x[8:-9]) # Extract only the ids from data
            talkid_dfs[f"{lang}"] = talkid_df
    return talkid_dfs
        
talkid_dfs = create_talkid_df(languages)

The number of talk ids in each languages: 
ja: 2489
en: 2683
de: 2161
ko: 2565
zh: 163
zh-cn: 2429
zh-tw: 2578
ar: 2545
fr: 2597


In [320]:
talkid_df_ja = talkid_dfs["ja"]
talkid_df_ja
talkid_df_de = talkid_dfs["de"]
talkid_df_de
talkid_df_en = talkid_dfs["en"]
talkid_df_en
talkid_df_ko = talkid_dfs["ko"]
talkid_df_ko
talkid_df_zh_cn = talkid_dfs["zh-cn"]
talkid_df_zh_cn
talkid_df_ar = talkid_dfs["ar"]
talkid_df_ar
talkid_df_fr = talkid_dfs["fr"]

In [362]:
with open(f'/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_de.xml', 'r', encoding="utf-8") as f: # , 
            data = f.read()
            bs_data = BeautifulSoup(data, "xml")
for e in bs_data.find_all('head'):
    if e.find('talkid') == None:
        #if len(e.find{'content'}) > 1: 
        print(e)

<head>
<url>http://www.ted.com/talks/roger_antonsen_math_is_the_hidden_secret_to_understanding_the_world?language=de</url>
<encoding>UTF-8</encoding>
<content-type>text/html; charset=utf-8</content-type>
<description>TED Talk Subtitles and Transcript: Entschlüsseln Sie die Geheimnisse und das Innenleben der Welt mithilfe einer der fantasievollsten Kunstformen: Mathematik. Roger Antonsen erläutert, wie ein leichter Perspektivenwechsel Muster, Zahlen und Formeln aufdecken und als Zugang zu Empathie und Verständnis dienen kann.</description>
<title>Roger Antonsen: Mathematik: das verborgene Geheimnis zum Verständnis der Welt</title>
<transcription>
<seekvideo id="1">Hallo. Ich möchte über Verständnis, die Eigenart und das Wesen von Verständnis sprechen, denn wir alle bemühen uns, zu verstehen. Wir wollen Dinge verstehen. Ich behaupte, dass Verständnis mit der Fähigkeit zusammenhängt, die Perspektive zu wechseln. Wenn man das nicht hat, kann man nicht verstehen. Das behaupte ich jedenfalls

In [68]:
def create_content_df(languages):
    content_dfs = {}
    print ("The number of content data in each languages: ")
    for lang in languages:
        content_df = pd.read_xml(f'/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_{lang}.xml', parser='lxml')
        content_df = content_df.drop(["head", "id"], axis = 1)
        print (f"{lang}: ", len(content_df))
        content_dfs[f"{lang}"] = content_df
    return content_dfs
    
content_dfs = create_content_df(languages)

The number of content data in each languages: 
ja:  2490
en:  2683
de:  2163
ko:  2565
zh:  163
zh-cn:  2429
zh-tw:  2578
ar:  2545
fr:  2598


In [198]:
content_df_ja = content_dfs["ja"]
content_df_ja 

,content
0,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...
2485,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [322]:
content_df_ja = content_dfs["ja"]
content_df_ja
content_df_de = content_dfs["de"]
content_df_de
content_df_en = content_dfs["en"]
content_df_en
content_df_ko = content_dfs["ko"]
content_df_ko
content_df_zh_cn = content_dfs["zh-cn"]
content_df_zh_cn
content_df_ar = content_dfs["ar"]
content_df_ar
content_df_fr = content_dfs["fr"]

In [340]:
def concatenate_df(df1, df2, df1_col, df2_col):
    
    if df1.isnull().any()[df1_col]:
        print ("NAN for df1")
        df1[df1_col] = df1[df1_col].replace({np.NaN: "no contents"})
        print (df1.isnull().any(), "for df1")
        print (df1)
        
    if df2.isnull().any()[df2_col]:
        print ("NAN for df2")
        df2[df2_col] = df2[df2_col].replace({np.NaN: "no contents"})
        
        
    if len(df1) == len(df2): 
        concat_df = pd.concat([df1, df2], axis=1)
        print (concat_df.isnull().any())

        
    elif len(df1) > len(df2):
        #print ("(df1.duplicated() == False)", df1[df1[df1_col].duplicated() == False], "df1[df1_col]=='no contents'", df1[df1[df1_col]=="no contents"])
        df1 = df1[(df1[df1_col].duplicated() == False) | (df1[df1_col]=="no contents")]
        print ("df2for NAN", df2.isnull().any()[df2_col])
        print ("df1for NAN", df1.isnull().any()[df1_col])
        print (df1, df2)
        print ("df1.duplicated", df1[df1["content"].duplicated()])
        df1.reset_index(drop=True, inplace = True)
        concat_df = pd.concat([df1, df2], axis=1)
        print (concat_df.isnull().any())
    
    """
    elif len(df1) < len(df2):
        df2 = df2[(df2[df2_col].duplicated() == False) & (df2[df2_col].isnull()==False)] # Remove the duplicated content 
        concat_df = pd.concat([df1, df2], axis=1)
        print (concat_df.isnull().any())
    """
    
    return df1, df2, concat_df



In [327]:
_, _, concat_df_ja = concatenate_df(content_df_ja, talkid_df_ja, "content", "talk_id")
concat_df_ja

df2for NAN False
df1for NAN False
                                                content
0     人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1     胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2     皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3     （ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4     文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...                                                 ...
2485  (拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486  おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487  どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488  (♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...
2489  ご来場の皆さん 本日はありがとうございます 皆さんなら 長期的視野に立った都市開発が 人類を...

[2489 rows x 1 columns]      talk_id
0      12908
1      13062
2      11327
3      12459
4      12346
...      ...
2484      96
2485      66
2486       1
2487       7
2488      53

[2489 rows x 1 columns]
content    False
talk_id    False
dtype: bool


,content,talk_id
0,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...,12908
1,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...,13062
2,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...,11327
3,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...,12459
4,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...,12346
...,...,...
2484,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...,96
2485,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...,66
2486,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...,1
2487,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...,7


In [328]:
_, _, concat_df_ko = concatenate_df(content_df_ko, talkid_df_ko, "content", "talk_id")
concat_df_ko

content    False
talk_id    False
dtype: bool


,content,talk_id
0,"""들어보렴. 들리니?"" 제 할머니가 물으셨죠. ""들어보렴. 딱정벌레가 뭐라고 하는지...",13591
1,마름병이 밤나무를 멸종시킬까요? 농부들은 대부분 아니라고 생각합니다. 마름병은 뿌리...,13062
2,"최근 우리는 사이버 공격이 비즈니스 세계에 미치는 영향을 봤습니다. JP 모건, 야...",12501
3,(음악)\n(노래) 오오 오오\n그녀는 부자가 되고 싶어요\n그녀는 유명해지고 싶죠...,12803
4,"2014년 6월 13일 평소와 같은 금요일 라이베리아의 수도, 몬로비아의 구원 병원...",12302
...,...,...
2560,(박수) 감사합니다. 저는 지금 기쁘기도 하면서 새로운 도전을 하고있는 기분이라고 ...,96
2561,안녕하세요. 정말 좋지 않습니까? 강연들이 너무 좋아 정신을 못 차리겠네요. 사실 ...,66
2562,"감사합니다, 크리스. 이곳에 두 번이나 설 기회를 얻게된 것을 큰 영광으로 생각합니...",1
2563,"♫안녕 자동응답기, 나의 오랜 친구여 (웃음) 기술 지원이 필요해서 또 전화했어. ...",7


In [341]:
_, _, concat_df_de = concatenate_df(content_df_de, talkid_df_de, "content", "talk_id")
concat_df_de # After 924, which is index 1475, Lassen Sie~ doesnt have talkid

df2for NAN False
df1for NAN False
                                                content
0     Ich wuchs auf als weißer Mann, säkular, in ein...
1     Treppen sind wahrscheinlich eine der emotional...
2     Es gibt keine schlechen Knöpfe, bloß schlechte...
3     Ich bin Wissenschaftlerin, und ein großer Star...
4     Wissen Sie, auf wen ich neidisch bin? Auf Leut...
...                                                 ...
2158  (Applaus) Vielen Dank. Ich muss Ihnen sagen, i...
2159  Guten Morgen. Wie geht es Ihnen?\n(Lachen)\nEs...
2160  Vielen Dank, Chris. Es ist mir wirklich eine E...
2161  ♪ Hallo Sprachbox, alter Freund ♪ (Lachen) ♪ I...
2162  Wenn Sie heute hier sind, und ich freue mich, ...

[2162 rows x 1 columns]      talk_id
0      12303
1      12352
2      12351
3       9930
4      10550
...      ...
2156      96
2157      66
2158       1
2159       7
2160      53

[2161 rows x 1 columns]
df1.duplicated           content
826   no contents
1055  no contents
content    False
ta

,content,talk_id
0,"Ich wuchs auf als weißer Mann, säkular, in ein...",12303
1,Treppen sind wahrscheinlich eine der emotional...,12352
2,"Es gibt keine schlechen Knöpfe, bloß schlechte...",12351
3,"Ich bin Wissenschaftlerin, und ein großer Star...",9930
4,"Wissen Sie, auf wen ich neidisch bin? Auf Leut...",10550
...,...,...
2157,"(Applaus) Vielen Dank. Ich muss Ihnen sagen, i...",66
2158,Guten Morgen. Wie geht es Ihnen?\n(Lachen)\nEs...,1
2159,"Vielen Dank, Chris. Es ist mir wirklich eine E...",7
2160,"♪ Hallo Sprachbox, alter Freund ♪ (Lachen) ♪ I...",53


In [330]:
_, _, concat_df_fr = concatenate_df(content_df_fr, talkid_df_fr, "content", "talk_id")
concat_df_fr

df2for NAN False
df1for NAN False
                                                content
0     Je suis un immigrant venu de l'Ouganda vivant ...
1     [Provocation de Danny Hillis :]\n[Il est grand...
2     Mon voyage pour m'éloigner de l'extrémisme vio...
3     « Le mildiou tuera-t-il le châtaignier ? » Les...
4     Il est 4 heures du matin. Je me réveille dans ...
...                                                 ...
2593  (Applaudissements) Merci. Je dois vous dire qu...
2594  Bonjour. Comment allez-vous? C'était bien, n'e...
2595  Merci beaucoup, Chris. C'est vraiment un honne...
2596  Bonjour répondeur, mon vieil ami. (Rires) J'ap...
2597  Si vous êtes là aujourd'hui — et j'en suis trè...

[2598 rows x 1 columns]      talk_id
0      14392
1      12799
2       8298
3      13062
4       9982
...      ...
2592      96
2593      66
2594       1
2595       7
2596      53

[2597 rows x 1 columns]
content    False
talk_id     True
dtype: bool


,content,talk_id
0,Je suis un immigrant venu de l'Ouganda vivant ...,14392
1,[Provocation de Danny Hillis :]\n[Il est grand...,12799
2,Mon voyage pour m'éloigner de l'extrémisme vio...,8298
3,« Le mildiou tuera-t-il le châtaignier ? » Les...,13062
4,Il est 4 heures du matin. Je me réveille dans ...,9982
...,...,...
2593,(Applaudissements) Merci. Je dois vous dire qu...,66
2594,"Bonjour. Comment allez-vous? C'était bien, n'e...",1
2595,"Merci beaucoup, Chris. C'est vraiment un honne...",7
2596,"Bonjour répondeur, mon vieil ami. (Rires) J'ap...",53


In [331]:
_, _, concat_df_ar = concatenate_df(content_df_ar, talkid_df_ar, "content", "talk_id")
concat_df_ar

content    False
talk_id    False
dtype: bool


,content,talk_id
0,هل حصل وذكرتَ دراسةً ما كمرجع لكلامك وأنت في ا...,12908
1,يقول الناس أنني شخصاً لطيفاً... إلى الحد حيثما...,12354
2,أنحدر من عائلة مكونة من خمسة إخوة، جميعهم علما...,3618
3,"أتذكر أنني فكرت -لنفسي- ، ""هذا سيغير كل شيء عن...",12353
4,أعتقد أن الدرج قد يكون واحدًا من العناصر الماد...,12352
...,...,...
2540,(تصفيق) شكراً. علي أن أخبركم بأنني أشعر بالتحد...,96
2541,صباح الخير. كيف حالكم؟ كان يوما عظيما، أليس كذ...,66
2542,شكرا جزيلا كريس. انه فعلا شرف عظيم لي ان أصعد ...,1
2543,أهلا بك صديقي العزيز، البريد الصوتي♪ (ضحك) لقد...,7


In [332]:
_, _, concat_df_zh_ch = concatenate_df(content_df_zh_cn, talkid_df_zh_cn, "content", "talk_id")
concat_df_zh_ch

NAN for df1
content    False
dtype: bool for df1
                                                content
0     我自认为既是一位艺术家， 又是一位设计师。 我在一个研究 人工智能的实验室工作。 我们正在尝...
1     我是一个编剧兼导演 写一些社会变革的故事， 因为我相信故事能够 打动并驱动我们。 故事使我们...
2     我记得曾经这样想过， “这将会颠覆我们交流的方式。”\n【小发明】 【大点子】\n【超链接:...
3     我认为台阶可能是 建筑家需要去解决的在情感上 最具可塑性的物理元素之一。\n[小事物，大想法...
4     没有不好的扣子，只有不好的人。 那听起来真么样？可以吗？\n“小物件” “大想法”\n“Is...
...                                                 ...
2424  （掌声） 谢谢。 我必须告诉你们我同时感到挑战与兴奋。 让我感到兴奋的是，我有一个付出回馈的...
2425  早上好，还好吗? 很好吧，对不对? 我已经飘飘然了! 我要飘走了。(笑声) 这次会议有三个主...
2426  非常谢谢，克里斯。的确非常荣幸 能有第二次站在这个台上的机会，我真是非常感激。 这个会议真是...
2427  语音信箱 我的老朋友♪ (笑声) 我又打了技术支持电话♪ 我没听老板的警告 在周一早上打的♪...
2428  假若你今天有幸来此—我对你的到来亦是甚喜－－ 你们一定听说过可持续发展可以拯救我们的生活 然...

[2429 rows x 1 columns]
content    False
talk_id    False
dtype: bool


,content,talk_id
0,我自认为既是一位艺术家， 又是一位设计师。 我在一个研究 人工智能的实验室工作。 我们正在尝...,12498
1,我是一个编剧兼导演 写一些社会变革的故事， 因为我相信故事能够 打动并驱动我们。 故事使我们...,11997
2,我记得曾经这样想过， “这将会颠覆我们交流的方式。”\n【小发明】 【大点子】\n【超链接:...,12353
3,我认为台阶可能是 建筑家需要去解决的在情感上 最具可塑性的物理元素之一。\n[小事物，大想法...,12352
4,没有不好的扣子，只有不好的人。 那听起来真么样？可以吗？\n“小物件” “大想法”\n“Is...,12351
...,...,...
2424,（掌声） 谢谢。 我必须告诉你们我同时感到挑战与兴奋。 让我感到兴奋的是，我有一个付出回馈的...,96
2425,早上好，还好吗? 很好吧，对不对? 我已经飘飘然了! 我要飘走了。(笑声) 这次会议有三个主...,66
2426,非常谢谢，克里斯。的确非常荣幸 能有第二次站在这个台上的机会，我真是非常感激。 这个会议真是...,1
2427,语音信箱 我的老朋友♪ (笑声) 我又打了技术支持电话♪ 我没听老板的警告 在周一早上打的♪...,7


In [333]:
_, _, concat_df_en = concatenate_df(content_df_en, talkid_df_en, "content", "talk_id")
concat_df_en

NAN for df1
content    False
dtype: bool for df1
                                                content
0     I will never forget the first time I visited a...
1     Don't you love a good nap?\n(Laughter)\nJust s...
2     So, I'll be speaking to you using language ......
3     All right, let's get ready for the worst TED T...
4     [Ojibwe: Hello. My English name is Tara; my Na...
...                                                 ...
2678  Thank you. I have to tell you I'm both challen...
2679  Good morning. How are you?\n(Laughter)\nIt's b...
2680  Thank you so much, Chris. And it's truly a gre...
2681  (Music: "The Sound of Silence," Simon & Garfun...
2682  If you're here today — and I'm very happy that...

[2683 rows x 1 columns]
content    False
talk_id    False
dtype: bool


,content,talk_id
0,I will never forget the first time I visited a...,14471
1,Don't you love a good nap?\n(Laughter)\nJust s...,13022
2,"So, I'll be speaking to you using language ......",12727
3,"All right, let's get ready for the worst TED T...",13778
4,[Ojibwe: Hello. My English name is Tara; my Na...,12800
...,...,...
2678,Thank you. I have to tell you I'm both challen...,96
2679,Good morning. How are you?\n(Laughter)\nIt's b...,66
2680,"Thank you so much, Chris. And it's truly a gre...",1
2681,"(Music: ""The Sound of Silence,"" Simon & Garfun...",7


# Pandas Dataframe

In [7]:
from datasets import load_dataset
import pandas as pd
from bs4 import BeautifulSoup

In [162]:
df_ja = pd.read_xml('/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_ja.xml', parser='lxml')

In [163]:
df_ja

,id,head,content
0,1,NaN,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,2,NaN,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,3,NaN,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,4,NaN,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,5,NaN,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...,...,...
2485,2495,NaN,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,2496,NaN,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,2497,NaN,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,2498,NaN,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [59]:
df_ja["content"].duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2485    False
2486    False
2487    False
2488    False
2489    False
Name: content, Length: 2490, dtype: bool

In [101]:
df_ja[df_ja["content"].duplicated()]


,id,head,content
699,701,NaN,私はこの夏長い旅に出ており そこで素晴らしい経験をしました イザベル・ウィルカーソンの名著 ...
1494,1497,NaN,None
1532,1536,NaN,None
1594,1598,NaN,None
1623,1627,NaN,None
1686,1690,NaN,None
1731,1735,NaN,None
1767,1771,NaN,None
2169,2177,NaN,None
2350,2359,NaN,None


In [171]:
df_ja

,id,head,content
0,1,NaN,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,2,NaN,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,3,NaN,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,4,NaN,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,5,NaN,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...,...,...
2485,2495,NaN,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,2496,NaN,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,2497,NaN,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,2498,NaN,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [180]:
#df_ja[df_ja["content"].duplicated() == False]

df_removed = df_ja[(df_ja["content"].duplicated() == False) | (df_ja["content"].isnull()==True)]
df_removed
#df_ja[(df_ja["content"] != np.NaN)]
#df_ja[df_ja["content"].duplicated()]
#df_ja[df_ja["content"].isnull()]
#df_ja = df_ja.drop(df_ja[df_ja[df1_col].duplicated()!="no contents"], axis=0)
#df_ja[df_ja["content"].isnull()==False]

,id,head,content
0,1,NaN,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,2,NaN,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,3,NaN,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,4,NaN,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,5,NaN,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...,...,...
2485,2495,NaN,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,2496,NaN,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,2497,NaN,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,2498,NaN,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [29]:
df_ja[698:700]
        

,id,head,content
698,700,NaN,私はこの夏長い旅に出ており そこで素晴らしい経験をしました イザベル・ウィルカーソンの名著 ...
699,701,NaN,私はこの夏長い旅に出ており そこで素晴らしい経験をしました イザベル・ウィルカーソンの名著 ...


File id 700 does not have talk id, so delete file 700 and all the empty content

In [15]:
df_ja[902:905]

,id,head,content
902,904,NaN,今日 皆さんにお話したいのは 私たちのあり方を決める パワフルで 基礎的なもの― 「声」につ...
903,905,NaN,None
904,906,NaN,まず 皆さんがスーパーにいると 想像して下さい 食料品を買うと 選択肢が与えられます プラス...


In [16]:
df_ja = pd.read_xml('/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_ja.xml', parser='lxml')
df_ja= pd.concat([talk_id_ja, df_ja], axis=1)
#df_ja["talk_id"][:-1] = df_ja["talk_id"][:-1].map(lambda x:int(x))
df_ja['talk_id'].isnull()
df_ja

,talk_id,id,head,content
0,12908,1,NaN,人と話をしている時に 言及した論文を 実際には読んでなかった ということはありませんか？\n...
1,13062,2,NaN,胴枯病は栗の木を 絶えさせるのか 農夫らはそうは思わない 根でくすぶりながら 新たな芽を送り...
2,11327,3,NaN,皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成...
3,12459,4,NaN,（ギター演奏）\n探し始めて\nもう何日にもなる\nどこに行ったの？\nどこに行ったんだろう...
4,12346,5,NaN,文明の歴史は ある意味 地図の歴史です 私たちは まわりの世界を どのように理解してきたので...
...,...,...,...,...
2485,66,2495,NaN,(拍手) 私は困難と興奮の 両方を感じています 興奮は この場で恩返し ができるからで 困難...
2486,1,2496,NaN,おはようございます。気分はいかがですか？素晴らしいですね、ここは すべてが驚嘆の連続です だ...
2487,7,2497,NaN,どうもありがとう クリス このステージに立てる機会を 2度もいただけるというのは実に光栄なこ...
2488,53,2498,NaN,(♪「サウンドオブサイレンス」の節で) やあボイスメール 懐かしき友よ (笑) またテクニカ...


In [17]:
df_en = pd.read_xml('/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt17/FILTERED_xml/ted_en.xml', parser='lxml')
df_en= pd.concat([talk_id_en, df_en], axis=1)
#df_en["talk_id"]= df_en["talk_id"].map(lambda x:int(x))
df_en[df_en["content"].isnull()]
df_en[df_en["id"].isnull()]

,talk_id,id,head,content


In [25]:
df_en["content"][902:905]

902    I came here to show you the Fotokite. It's a t...
903    Let me tell you a story about a little girl na...
904    George and Charlotte Blonsky, who were a marri...
Name: content, dtype: object

In [92]:
df_ja["content"][2]

'皆さんも多分 偉大な人物が いかにして偉業を成し遂げたのかと 考えたことがあると思います 成果が際立っていればいるほど 私たちは その人を天才と呼んだり もしかすると他の惑星から来た 異星人に違いないなどと言い 自分とは まったく違う存在だと 思いがちです でも そうなのでしょうか？\n１つの例から始めましょう ニュートンのリンゴの話は 知っていますよね？ 本当なのでしょうか？ 多分違うでしょう それでも 「リンゴ」が何もなかったとは 考えにくいです つまり 何か足掛かりや 特定の状況があって 万有引力の着想が不可能ではない 状態にあったということです そして間違いなく ニュートンにとっては 不可能ではありませんでした 可能だったのです それも 何かの理由で ある時点において それはリンゴのように 容易に取れるものだったのです それこそがリンゴなのです\nアインシュタインはどうでしょう？ 相対性理論は アイデアの歴史における もうひとつの大きな飛躍であり 彼以外には 着想できない ものだったのでしょうか？ あるいはむしろ それは手の届くところにある 可能なものだったのか— もちろんアインシュタインにとっては ということですが— それは彼独自の科学の道における 小さなステップだったのか？ もちろん私たちには そのような道は見出せないでしょうが かと言って そんな道が 存在しないとは言い切れません\nこれは とても 示唆には富んでいますが まだ確かとは言い難いでしょう 素晴らしいアイデアがどこから来るのか あるいは 一般に 新しいものが いかにもたらされるのか 知りたいと思うのであれば 物理学者として 科学者として 私が学んだことは 解の半分は 適切な問いを 立てることにあるということですが 私たちは今や 適切な問いを 見付け 取り組むための 素晴らしい概念的な枠組みを 手にしつつあると思います そこで既知の領域の境界まで 皆さんをお連れして— まあ 私が知る 限りの境界ですが 既知の事柄が 強力で素晴らしい出発点となり 新規性や イノベーション 創造性といった言葉の 深い意味の把握に繋がることを 示しましょう\n「新しいもの」について議論しますが もちろん その背後の 科学にも触れます 新しいものが私たちの生活に 取り込まれる方法は色々あり それは ごく個人的なもの

Question:
- Can we sort xml file by talk_id? or can we extract talk_id info in pandas too?  

In [3]:
df = pd.read_csv("/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/multilingual_talks_dict", index_col=0)

In [5]:
df.transpose()

,en,ar,fr,zh,ko,de,ja
talk_id 1553,['There are a lot of ways the people around us...,['هناك الكثير من الطرق التي يمكن بها للناس من ...,['Les gens autour de nous peuvent contribuer à...,"['我们周围的人能以很多种方式 来帮我们把生活变得更美好', '我们不一定能常常碰到我们的邻...",['우리 주변의 사람들이 우리의 인생을 향상시키기 위해 도울 수 있는 방법은 여러가...,"['Es gibt viele Möglichkeiten, wie unsere Mitm...","['まわりの人は様々な形で 人生を豊かなものに してくれるものですが', 'でも同じ場所に ..."
talk_id 1144,"['My name is Amit.', ""And 18 months ago, I had...","['اسمي هو أميت.', 'وقبل 18 شهرا، كنت أشغل وظيف...","[""Je m'appelle Amit."", ""Il y a 18 mois, je fai...","['我叫阿密特', '18月以前，我在谷歌有另外一个工作 我跟我的老板提出了这个点子 想要在...","['제 이름은 아미트입니다.', '18개월 전, 저는 구글에 입사했습니다. 그리고 ...","['Mein Name ist Amit.', 'Vor 18 Monaten machte...","['私はアミットと言います', '1年半前 Googleで 別な仕事をしていたときに 美術館..."
talk_id 1165,"['In 2007, I decided that we needed to reconc...",['في 2007 قررت اننا نحتاج لاعادة النظر في المف...,"[""En 2007, J'ai décidé que nous devions repens...","['2007年 我觉得我们需要重新定义 我们意识中的经济发展的概念。', '我们新目标应该致...","['2007년에 저는 경제 개발에 대해서 우리가 생각했던 방식을', '재개념화할 필...","['Im Jahr 2007 habe ich beschlossen, dass wir ...","['2007年 私は 経済成長について改めて概念化し直す 必要があると思い立ちました', '..."
talk_id 1898,"[""Hello, TEDWomen, what's up."", 'Not good enou...","['مرحبا TEDWomen كيف حالكم؟', 'هذا ليس كافيا.'...","['Bonjour, TEDWomen, ça va ?', ""C'est pas enco...","['各位女同胞们，大家好！ 你们还好吗？', '（欢呼声） 再大声点', '各位女同胞们，大...","['안녕하세요, TEDWomen 참석자 여러분! 안녕하세요!', '충분하지 않군요....","['Hallo TEDWomen, was geht?', 'Das reicht nich...","['こんにちは TEDWomenの皆さん！', '声が小さいですね', 'こんにちは TED..."
talk_id 1520,['with this photo of a bullet piercing through...,"['ألهمنا ""دوك ادغرتون"" بالرهبة والفضول بهذه ال...","[""Doc Edgerton nous a impressionnés et intrigu...",['爱哲顿博士用这张子弹击穿苹果的照片 激起了我们的敬佩和好奇， 这张照片的曝光时间只有一百...,['닥 에저톤 선생께서 총알이 사과를 뚫고 지나가는 이 사진으로 경외감과 호기심으로...,['Doc Edgerton hat uns mit Ehrfurcht und Neugi...,"['エジャートン博士の リンゴを撃ちぬく弾丸の写真は 100万分の1秒を捉えたもので', '..."
...,...,...,...,...,...,...,...
talk_id 1699,"['This is where I live. I live in Kenya, at t...","['هنا حيث أعيش.أعيش في كينيا, في الجهة الجنوبي...","[""C'est là que je vis. Je vis au Kenya, au sud...","['这是我所生活的地方，肯尼亚， 在内罗毕国家公园的南部', '在我的身后是我父亲的奶牛 在...","['이곳이 제가 사는 곳, 케냐 나이로비 국립공원의 남쪽입니다.', '저 뒤에 있는...",['Hier lebe ich. Ich lebe in Kenia am südliche...,"['僕は ここで暮らしてます ケニアの― ナイロビ国立公園の南部に住んでいます', '後ろに..."
talk_id 69,"['You know, one of the intense pleasures of tr...",['هل تعرفون أن أحد المتع الكبيرة للسفر وأحد مب...,"[""Vous savez, un des plaisirs intenses du voya...",['对我来说 在世界各地旅行 和从事人类学研究的生涯中 最大的乐趣之一莫过于 体验那些没有忘...,['여행의 가장 큰 즐거움 중 하나이자 민속지학적 연구의 기쁨 중 하나는 구식 방법...,"['Wissen Sie, eines der großen Vernügen beim R...",['旅の大きな楽しみの１つであり また民族誌学の調査をする喜びの１つは 昔ながらの暮らしを忘...
talk_id 1548,"['the Oxford English Dictionary is ""snollygost...",['واحدة من كلماتي المفضلة في قاموس أوكسفورد ال...,['Un des mots que je préfère dans le dictionna...,"['我很喜欢的词之一 在整个牛津英语词典里叫“snollygoster”。', '不仅因为它...","['옥스포드 영어 사전에서 제가 가장 좋아하는 단어는 ""악덕 정객""이라는 단어입니다...",['Eines meiner Lieblingswörter im gesamten Oxf...,"['オックスフォード英語辞典で', 'お気入りの単語は “snollygoster” です'..."
talk_id 1829,"[""So I'm going to talk about trust, and I'm g...",['إذن سأتحدّث اليوم عن الثّقة، و سأبدأ بتذكيرك...,"[""Je vais maintenant parler de confiance, et j...","['我今天要讲的是信任。 一开始，我要先向你们交代一下， 人们对信任的基本观点。', '我觉...","['자, 이제부터 신뢰에 대해서 말씀드리겠습니다. 먼저 신뢰에 대한 여러분들의 견해...",['Ich möchte über Vertrauen reden und ich möch...,['信頼についてお話しするわけですが まずは 信頼について 一般的に思われていることから 始...
